In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
        mage import transform
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
from segment_anything import sam_model_registry, build_sam_vit_b_multiclass
import torch.nn.functional as F
import random
from datetime import datetime
import pandas as pd
import nibabel as nib
import pickle

In [47]:
df = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/per_class_isolated_df/yolov7/path_df_pooled_labels_only_with_bbox_yolov7.csv')
df_all = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv')
df = df.drop(['index'], axis='columns')


In [48]:

df_all_tmp = pd.concat([df_all]*102,keys = list(range(1,103)),names = ['label_number']).reset_index(level=0)
df_all_tmp = df_all_tmp.drop(['bbox_0', 'bbox_1', 'bbox_2', 'bbox_3'], axis = 'columns')
df_tmp = df.drop(['image_embedding_slice_path', 'segmentation_slice_path', 'image_path'], axis='columns')
df_merged = df_all_tmp.merge(df_tmp, how = 'left', on = ['id', 'slice', 'label_number'])

In [9]:
dest_path = f'/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/yolov7_bboxes/{id}/{slice}.npy'

In [59]:

for i, id in enumerate(tqdm(np.unique(df_merged['id']))):
    id_df = df_merged[df_merged['id'] == id]
    
    for slice in range(256):
        slice_df = id_df[id_df.slice == slice]
        slice_df = slice_df.sort_values('label_number')
        arr = np.array([slice_df.bbox_0, slice_df.bbox_1, slice_df.bbox_2, slice_df.bbox_3]).T
        if not os.path.exists(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/yolov7_bboxes/{id}'):
            os.makedirs(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/yolov7_bboxes/{id}')
        dest_path = f'/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/yolov7_bboxes/{id}/{slice}.npy'
        np.save(dest_path, arr)

100%|██████████| 1113/1113 [14:30<00:00,  1.28it/s]


In [67]:
# make a path dataframe with this as the path
df_with_box_path = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df.csv')

df_with_box_path['box_path'] = df_with_box_path.apply(lambda x: f'/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/yolov7_bboxes/{x.id}/{x.slice}.npy', axis=1)

In [68]:
df_with_box_path.to_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_with_yolov7_box_path.csv')